## 28/2/2024

Solution data structure
- Solution
    - "nodes": Nodes solution
    - "solution": Solution values on solution nodes
    - "elements": Elements of solution nodes
    - "elementsBoundaryNodesOnly": Elements which only connect the boundary nodes and thus are more efficient for random point sampling
    - "areaElementsBoundaryNodesOnly": Area of the elements which only connect the boundary nodes
    - *Interpolation function for the solution for the encoder: interpSE(x,y)
    - *Interpolation function for the solution for the decoder: interpSD(x,y)
- Boundary Lines
    - List of vectors of cloed boundary curves nodes indexes: [idxs1[], idxs2[], idxs3[], …] (the first curve is the outer loop, the rest are inner holes) (first element is also the last)
    - List of matrices of nodes along the boundary curves: [nodesB1[], nodesB1[], nodesB1[], ...]
    - List of vectors of distance along the line: [distances1[], distances2[], distances3[], ...]
    - List of curve lengths per line: [L1, L2, L3, ...]
    - List of vectors of normals along the line [normals1[], normals2[], normals3[], ...]
    - *List of interpolation functions for the solution along the boundary lines: [interpBS1(distance), interpBS2(distance), interpBS3(length), …]
    - *List of interpolation functions for the normals along the boundary lines: [interpBN1(length), interpBN2(length), interpBN3(distance), …]
    - *List of interpolation functions for the x,y coordinates along the boundary lines: [interpBC1(distance), interpBC2(distance), interpBC3(distance), …]
    - *Interpolation function for isInsideTheDomian: interpD(x,y)
- Boundary Corners
    - List of corner nodes indexes: [idxNode1, idxNode2, idNode3, …]
    - List of the boundary curves that include the corner: [idxCurve1, idxCurve2, idxCurve3, …]
    - List of corner indexes within the curve: [idxCornerCurve1, idxCornerCurve1, idxCornerCurve1, ...]
    - List of distances where the corners are on their respective curves [distance1, distance2, distance3, ...]


\* Not part of saved dataset but computed when loading the dataset

The boundary line direction convention
- The domain is always to the right of the line if looking in the direction of the line
    - outer boundary edge is counter-clockwise
    - inner boundary edge is clockwise
- The boundary normals always point outwards from the domain

## 29/2/2024

For the boundary interpolation Pchip interpolator used as it does not introduce oscilations around discontinuities and is a high-order (cubic) method

## 1/3/2024

The low reynodls number flow data processed to the desirable formad and uploaded to TrainingData. REQUIRES TESTING!!!

## 4/3/2024

Paper structure
General Method description
1. Key idea description
    1. Describe the boundary observation problem
    2. Describe the 3 operator approach for solving it
2. Related methods review
3. General description of the training
4. General description of Encoder
5. General description of Boundary Encoder
6. General description of Decoder
7. General description of the elliptic system
Numerical Implementation
1. Detailed description of the Encoder
2. Detailed description of the Boundary Encoder
3. Detailed description of the Decoder
4. Detailed description of the elliptic system
5. Detailed description of the Training loop
    1. Training Patches Sampling
    2. Forward pass
    3. Loss and Optimisation
6. Detailed description of solving a boundary value problem

## 13/3/2024

The interior and boundatry training sample implemented with the updated version of the solution data structure which allows to handle the fluid examples

The low reyonolds number flow dataset has small but non-zero velocity values at the boundary. For now I left it as it is but possibly should be fixed in the future

There is an inefficiency in the current implementation in interpolating the interior encoder input u_IE and b_IE. the intepSE function is also sampling b_IE internally but not returning it. It would be way more efficient to just return b_IE from interpSE. Shoud change in future

## 24/3/2024

The updated training loop was implemented without the corner training patches and applied to th low reyonodls number flow dataset. The results are promising, the SINNs model is able to qualitatively model the flow reasonably well and there are wierd artefacts around corners. 
- This is likely caused by the fact that there are no training patches around the corners and so the model will not learn how to handle them. The fix is ADDING THE CORNER TRAINING PATCHES INTO THE TRAINING DATASETS
- Secondly there is a an attempt by the trained model at a smooth transition between the latent variables at boundary and in the interior of the domain but the transition has only the lenght of the interior encoder mask radius. On way to fix this could be adding x,y distances to the outer boundary / clossest boundary into the interior encoder input data.

IMPORTANT NOTE: I have from the beggining just used very small training patches but it might be worth exploring If I could just solve a larger mesh problem or even the entire mesh during the training. This might have a lof of benefits

## 27/3/2024

- I have implemented a cornern training patch dataset generation and use during the training but it does not seem to significantly improve the modeling accuracy after few test, I should investigate why the strange corner behaviour is still hapenning.
- I have also imporved the sampling process of boundary training patches which imporves its behavirou around corners. It worked but it made the sampling process very slow, I should look into why is it so slow and try to make it faster
- THERE MIGHT BE AN ISSUE WITH THE LINEAR SOLVER, it would explain why there is a weird artefact around the left bottom corner in both the solved latent variable and the resiludal of encoded latent variable. Maybe there is an incorrect source term on than node which creates this problem.

## 29/3/2024

- An issue was fixed with the linear FEM solver. It was caused by the new data format the the idxCurves is defided as an array which begins and ends with the same element. This duplicated some indexes in the array defining boundary nodes for the solver and confused it. It was fixed now and it made the model singificantly better
- It was found that an optimal normalization power is w=0.9. One issue which might exist with this normalization is that in normalizaes with variations around mean while for the second order PDEs it would be more appropriate to normalise with variation aroun the best fit plane. Calculating best-fit plane requires solving a 3x3 linear system which might slow down the training. It should be implemented and tried

## 3/4/2024

- The model works reasonably well and gives kvalitatively correct predictions of the flow field but it does not seem to be improving dramatically with the number of latent variables. I have studied how the loss is distributed across the domain and found that there are these few peaks which completaly dominate the loss and these peaks arrise very rapidly where the point in a very close area around them have significantly smaller loss. This seems like a weird inconsistency and should be investigated
- Significant imporvement has been achieved by increasing the boundary encoder input mask length. The likely explanation is that it gives the boundary encoder more information about the local boundary encoder and allows it to estimate things like gradient normal to the boundary which is uknown.
- A interesting experiment for the FYP could be seeing how the modeling accuracy imporves as the boundary encoder input mask length is increased.
- The model improves with this change but is still limited. The issue is likely the behaviour very close to the inner boundary where the mesh is very fine. The fine mesh behaviour is not represented in the training as the training patches have fixed size. I came up with 2 possible fixes:
    1. The mesh used to solve the latent BOP has uniform roughness everywhere which is the same as the one used for the training
    2. The size of the training patch is sampled based on the element size of the original mesh

## 10/4/2024

- Both mesh improvements listed above for the lowReynoldsNumber dataset listed above ((1)make solution mesh element size uniform and (2) make training patch mesh size variable based on the solution mesh) has been implemented.
    1. This does not seem to work at all. The modeling accuracy with a model trained this way is very low. 
        -Possible explanations:
            - The variance in training patch mesh size is very large and so the loss on the small training patches near the inner boundary can be smaller due to smaller area of elements. This means that the training will mainly focus on fitting the data to the training patches near the outer boundary. This will just exacurate the issue with low modeling accuracy around the inner boundary. 
        - Possible improvements:
            - Higher sampling rate around the inner boundary.
            - Better normalisation that would increase the loss around inner boundary compared to the outer boundary. Maybe normalising by R2 around best fit plane rather than variance. 
            - Maybe add a term in loss associanted with the residuals of the linear reconstruction.
    2. It seems to nor lead to significant imporvements but it is faster to compute.
- Very interesting finding is that if the training it run and evaluated with just one solution example the modeling accuracy can be very heigh. This means that the training architecutre can technically work it just struggle to generalise to the diversity of the data in the dataset for now. This should be further explored to understand if I can train very good models on signidicant portions of the dataset but maybe not all of them
- Conclusion: 
    - Exaplore training on subsets of the entire dataset
    - Try to figure out if there is some dissymmetry between training patches near inner and outer boundary. Can it be improved through better normalisation/sampling/loss

## 22/4/2024

- There seems to be a significant difference in using cubic vs laser interpolation for interpSE/interpSD. Could be an interesting experiment for the report

Experiments And Discussion:
- non-linear heat-transfer dataset:
    - Can learn approximately the weird analytical linearizing transformation 
    - Show that it can learn with very small number of datpoints
        - How good modeling performance can be achieved with N given solutions
- low reynolds number flow dataaset
    - show the convergence between the number of latent dimensions and modeling accuracy
    - show the relationship between the dataset variety (range of areas or something similar) and the modeling accuracy
    - relationship between the length of the boundary input mask and the modeling accuracy

## 02/11/2024

- started working on making the SINNs architecture dynamic by adding the transient term to elliptic system.
- could be interesting to see how the learned dynamic elliptic PDE can be used when solved for steady-state

## 24/11/2024
- The short term focus will be on apply the traditional BOP SINNs architecture on tubrbulent data
- Significant imporvement in convergence achieved by reducing the magnitude of the initial P (0.01*eye() works very well)
- Further imporvment from gausian filtering of the data along the x,y dirrection
- 4500 dataset:
    - meshing_size: 0.1 => too coarse
    - meshing_size = 0.07; n_latent = 8; fd_l_weight = 0.3; sizeIE = 1.2*meshing_size; sizeBE = 0.3; P_scaling = 0.01
        - increasing the fd_l_weight is not improving results